<h2 align="center">pflacs: Faster loadcases and parameter studies</h2> 
<h4 align="center">enhancing the engineering design process with Python</h4> 
<h3 align="center">Stephen McEntee</h3>  
<h3 align="center">PyConIE 2019-10-12</h3>


### about me ...

- chartered mechanical engineer
- subsea / pipeline engineer in oil & gas industry since 1997
- Python programming since ~2003
- my own startup ``Qwilka``
    - developing ``Visinum`` data management and analytics platform 
    - unstructured engineering data from engineering survey
    - lidar, sonar, video, images, etc.
- website: https://qwilka.github.io/
- Github: https://github.com/qwilka
- Linkedin: https://www.linkedin.com/in/stephen-mcentee-51a187121



![2019-09-30_Visinum_screenshot3.jpg](img/2019-09-30_Visinum_screenshot3.jpg)

![2019-09-30_Visinum_screenshot1.jpg](img/2019-09-30_Visinum_screenshot1.jpg)

## Scope

1. Motivation: automate engineering computations
1. Engineering: civil/mechanical/stuctural
1. Using Python to enhance the engineering design process
1. Introducing Python modules (open source, MIT licence):
    - ``pflacs`` faster load-cases and paramter studies
    - ``vntree`` tree data structure
    - ``PDover2t`` library of functions for pipelines design and analysis 


### The ``Barlow`` formula for hoop stress in a pressurised cylinder

$$\sigma_H = \frac{PD}{2t}$$  

<img src="img/hoop_stress_in_pipe_Elliotis_3.png" alt="hoop stress" title="Title text" style="display: inline-block; margin: 0" /> 
&nbsp;&nbsp;&nbsp;<small>image: 
<a href="https://creativecommons.org/licenses/by/3.0/">CC BY 3.0</a> 2013 <a href="https://www.researchgate.net/publication/290835187_A_Finite_Element_Approach_for_the_Elastic-Plastic_Behavior_of_a_Steel_Pipe_Used_to_Transport_Natural_Gas">Miltiades C. Elliotis</a>  
</small>

<h4 align="center">"pipeline engineering is nothing but PD/2t"</h4>

### Typical engineering design process (in brief)

- Establish the `Design Basis`
  - design parameters
  - constraints & assumptions
  - specify design codes and standards
- Plan analyses in hierarchical structure:
  - base-case
    - load cases
      - parameter studies
- Analyse in accordance with prescibed Design Code
  - Levels of analysis: 1, 2 and 3
- Final goal: compliance with design code
  - regulatory driven, not optimisation

## engineering design process workflow

- Iterative process (frequent re-work)
- Typical software tool chain:
  - spreadsheet (level 1 analysis)
  - computational worksheet (level 1 analysis)
  - finite element program (level 3 analysis)
  - word processor for reporting
- PC on every desk
  - GUI point-n-click environment
  - main productivity tool: copy-n-paste
- Manual paper-based workflows re-implemented on PCs

# Objectives

flexibility
light-touch framework
minimise restrictions

## Basic usage of `pflacs`

In [1]:
import pflacs

basecase = pflacs.Premise("Base case", 
                      parameters={"a":10, "b":5} )

print(f"basecase.a={basecase.a} basecase.b={basecase.b}")

basecase.a=10 basecase.b=5


#### `pflacs.Premise.plugin_func` used  to 'plugin' or patch a function

In [2]:
def add_nums(a, b, c=0):
    """Function adds 2 or 3 numbers, and returns the sum."""
    return a + b + c

In [3]:
basecase.plugin_func(add_nums)
basecase.add_nums

In [4]:
pflacs.Premise.add_nums

In [5]:
basecase.add_nums is add_nums

False

In [6]:
help(basecase.add_nums)

Help on Function in module __main__:

add_nums(a, b, c=0)
    Function adds 2 or 3 numbers, and returns the sum.



In [7]:
basecase.add_nums()

15

In [8]:
basecase.a + basecase.b == basecase.add_nums()

True

In [9]:
basecase.add_nums(b=-3)

7

In [10]:
basecase.a + (-3) == basecase.add_nums(b=-3)

True

In [11]:
basecase.b

5

In [12]:
basecase.add_nums(5, 4.02, -3)

6.02

In [13]:
5 + 4.02 + (-3) == basecase.add_nums(5, 4.02, -3)

True

In [14]:
print(f"basecase.a={basecase.a}, basecase.b={basecase.b}")

basecase.a=10, basecase.b=5


#### let's plugin another funcion

In [15]:
def sub_nums(x, y, z=0):
    """Function subtracts 2 or 3 numbers, and returns the result."""
    return x - y - z

In [16]:
basecase.plugin_func(sub_nums, argmap={"x":"a", "y":"b", "z":"c"} )

True

In [17]:
basecase.add_param("c", 6.5)

True

In [18]:
basecase.c

6.5

In [19]:
basecase.sub_nums()

-1.5

In [20]:
basecase.a - basecase.b - basecase.c == basecase.sub_nums()

True

`pflacs.Premise` is a tree

In [21]:
import vntree
issubclass(pflacs.Premise, vntree.Node)

True

let's make a new loadcase, based on `basecase`

In [22]:
lc1 = pflacs.Premise("Load case 1", parent=basecase, parameters={"a":100})
lc1.parent.name

'Base case'

In [23]:
print(basecase.to_texttree())

| Base case
+--| Load case 1



In [24]:
lc1.add_nums()

111.5

In [25]:
lc1.a + lc1.b + lc1.c == lc1.add_nums()

True

In [26]:
print(f"lc1.a = {lc1.a}, basecase.a = {basecase.a}")

lc1.a = 100, basecase.a = 10


In [27]:
print(f"lc1.b = {lc1.b}, basecase.b = {basecase.b}")

lc1.b = 5, basecase.b = 5


In [28]:
lc1.b is basecase.b

True

#### introducing `pflacs.Calc`

In [29]:
issubclass(pflacs.Calc, pflacs.Premise)

True

In [30]:
lc1_1 = pflacs.Calc("Load case 1-1 «sub_nums()»", lc1, funcname="sub_nums")
print(lc1_1._root.to_texttree(func=lambda n: f" {n.name} (node type: {n.__class__.__name__})"))

| Base case (node type: Premise)
+--| Load case 1 (node type: Premise)
.  +--| Load case 1-1 «sub_nums()» (node type: Calc)



In [31]:
lc1_1.a - lc1_1.b - lc1_1.c == lc1_1()

True

In [32]:
lc1_1._sub_nums

88.5

In [33]:
lc1_2 = pflacs.Calc("Load case 1-2 «add_nums()»", 
                    lc1, 
                    funcname="add_nums", 
                    argmap={"return":"add_nums_result"})
lc1_2()
lc1_2.add_nums_result

111.5

#### make a Pandas dataframe from the result of the `Calc` node execution

In [34]:
lc1_2.to_dataframe()   # dataframe stored in lc1_2._df

,a,b,c,add_nums_result
0,100,5,6.5,111.5


#### let's make a new loadcase

In [35]:
lc2 = basecase.add_child( lc1.copy() )  # method vntree.Node.copy clones a node or branch

#### changing the attribute values in the new nodes

In [36]:
lc2.name = "Load case 2"
lc2.a = 200
lc2_1 = lc2.get_child_by_name("Load case 1-1 «sub_nums()»")  # using vntree.Node.get_child_by_name
lc2_1.name = "Load case 2-1 «sub_nums()»»"
lc2_2 = lc2.get_child_by_name("Load case 1-2 «add_nums()»")
lc2_2.name = "Load case 2-2 «add_nums()»"

#### Let's plugin another function

In [37]:
def multipily_xyz(k:"a", l:"b", m:"c" = 1) -> "mult_nums_result":
    """Function multiplies 2 or 3 numbers, and returns the product."""
    return k * l * m

basecase.plugin_func(multipily_xyz, newname="mult_nums")

True

Using function annotations instead of `pflacs.Premise.plugin_func` argument `argmap` to re-map argument names and return attribute

"Python does not attach any particular meaning or significance to annotations"

"annotation consumers can do anything they want with a function's annotations"

[PEP 3107](https://www.python.org/dev/peps/pep-3107/#id29)

let's create another load case with `pflacs.Calc`

In [38]:
lc3 = pflacs.Calc("Load case 3 «mult_nums»", basecase, funcname="mult_nums")
import numpy
lc3.b = numpy.linspace(0,10,5)
lc3()
lc3.to_dataframe()

,a,b,c,mult_nums_result
0,10,0.0,6.5,0.0
1,10,2.5,6.5,162.5
2,10,5.0,6.5,325.0
3,10,7.5,6.5,487.5
4,10,10.0,6.5,650.0


#### Let’s take a look at the tree structure of the study we have built:

In [39]:
print(basecase.to_texttree())

| Base case
+--| Load case 1
.  +--| Load case 1-1 «sub_nums()»
.  .  | Load case 1-2 «add_nums()»
.  | Load case 2
.  +--| Load case 2-1 «sub_nums()»»
.  .  | Load case 2-2 «add_nums()»
.  | Load case 3 «mult_nums»



Each `vntree.Node` instance is a generator, so the tree can be traversed simply by interating over the root node. 
To re-execute the Calc nodes:

In [40]:
for node in basecase:
    if type(node) == pflacs.Calc:
        result = node()
        print(f"{node.name} calculated {result}")

Load case 1-1 «sub_nums()» calculated 88.5
Load case 1-2 «add_nums()» calculated 111.5
Load case 2-1 «sub_nums()»» calculated 188.5
Load case 2-2 «add_nums()» calculated 211.5
Load case 3 «mult_nums» calculated [  0.  162.5 325.  487.5 650. ]


Now that our study has been re-calculated, we will save it as `Pickle` file:

In [41]:
basecase.savefile("basic_usage_example.pflacs")

True

To re-open the study, we would use the class method `pflacs.Premise.openfile` that is inherited from `vntree.Node` 

In [42]:
new_study = pflacs.Premise.openfile("basic_usage_example.pflacs")
print(new_study.to_texttree())

| Base case
+--| Load case 1
.  +--| Load case 1-1 «sub_nums()»
.  .  | Load case 1-2 «add_nums()»
.  | Load case 2
.  +--| Load case 2-1 «sub_nums()»»
.  .  | Load case 2-2 «add_nums()»
.  | Load case 3 «mult_nums»



#### For large projects, saving results in a `Pickle` file could be inconvenient, so the results from `pflacs.Calc` nodes can be saved in a `HDF5` file:

In [43]:
for node in basecase:
    if type(node) == pflacs.Calc:
        node.to_hdf5()

![basic_usage_example_HDFView](img/basic_usage_example_HDFView.jpg)

# the end...

### thanks very much